In [ ]:
import random
from uuid import uuid4

from aleph_alpha_client import Prompt
from dotenv import load_dotenv

from intelligence_layer.connectors import StudioClient
from intelligence_layer.core import (
    CompleteInput,
    InMemoryTracer,
    LuminousControlModel,
    Task,
    TaskSpan,
)

load_dotenv()

# How to log and debug a task
The Intelligence Layer offers logging and debugging via a `Tracer`.  
Here are several steps you can use to debug tasks with the trace feature:

-----
Most logging of a task (input, output, time) is done simply by inheriting from `Task`. This logs to a trace.

 - If you don't care about logging and tracing, use the `NoOpTracer`.
 - To create custom logging messages in a trace use `task_span.log()`.
 - To map a complex execution flow of a task into a single trace, pass the `task_span` of the `do_run` to other execution methods (e.g. `Task.run()` or `model.complete()`). 
   - If the execution method is not provided by the intelligence layer, the tracing of input and output has to happen manually. See the implementation of `Task.run()` for an example.
 - Use the [submit trace functionality of the `StudioClient`](./how_to_use_studio_with_traces.ipynb) to view and inspect a trace in Studio

### Example

In [ ]:
class DummyTask(Task[str, str]):
    def __init__(self, model: LuminousControlModel | None = None) -> None:
        self._model = model if model else LuminousControlModel()

    def do_run(self, input: str, task_span: TaskSpan) -> str:
        should_output = random.random()
        # log a custom message and value
        task_span.log(
            "My very important log message that logs a random value", should_output
        )
        if should_output > 0.5:
            model_input = CompleteInput(prompt=Prompt.from_text(input), temperature=0.2)
            # Create a trace tree by passing task_span to .run or .complete methods.
            completion = self._model.complete(model_input, task_span)
            return completion.completions[0].completion
        else:
            return "Nope!"


tracer = InMemoryTracer()
DummyTask().run("", tracer)

project_name = str(uuid4())
studio_client = StudioClient(project=project_name)
my_project = studio_client.create_project(project=project_name)

submitted_trace_id = studio_client.submit_from_tracer(tracer)


pass